In [1]:
import numpy as np
import pandas as pd
import matplotlib.dates as mdates

In [2]:
def tau_x_y(wind_speed,component):
    temp_list = np.zeros(wind_speed.shape)
    for i in range(len(wind_speed)):
        if wind_speed[i]<11:
            temp = (1.3*1.2e-3)*wind_speed[i]*component[i] # EW surface wind stress for low wind speed
            temp_list[i]=temp
        else:
            temp = (1.3*1.2e-3)*(0.49+0.065*wind_speed[i])*wind_speed[i]*component[i] # EW surface wind stress for high wind speed
            temp_list[i]=temp     
    return temp_list

In [3]:
df = pd.read_csv("met_data_svalbard_lufthavn.csv",delimiter=";",skipfooter=1,na_values='-',engine="python")
df.drop(columns=["Station"])
df['Time'] = pd.to_datetime(df['Time'], format='%d.%m.%Y %H:%M')

df['Wind direction'] = df['Wind direction'].interpolate()

## Wind component conversion from met-coordinates (0 deg from N) to wind stress
## Calculate wind stress based on Large & pond (1981)


In [4]:
df["u10"] = -df["Mean wind speed"]*np.sin(df["Wind direction"]/360*2*np.pi)
df['taux'] = tau_x_y(df['Mean wind speed'],df['u10'])
df["v10"] = -df["Mean wind speed"]*np.cos(df["Wind direction"]/360*2*np.pi)
df['tauy'] = tau_x_y(df['Mean wind speed'],df['v10'])

## Calculate wind stress based on Large & pond (1981)

In [5]:
rhoa = 1.3           # Air density in kg/m3
C    = 2.0e-3        # Heat transfer coefficient (sensible + latent heat flux including evaporation effects)
cp   = 1004          # Specific heat at constant pressure for dry air in J/(deg kg)
Ts   = -1.865         # Ocean surface temperature, assumed constant. -1.865 = freezing point for surface seawater with salinity Sw = 34.
es   = 0.98          # Sea surface emissivity
sig  = 5.67e-8       # Stefan-Boltzmann constant
alpha= 0.1           # Albedo of open water
S0   = 1353          # The sun constant in W/m2
psi  = 77.75         # Latitude of the polynya in Degrees
r    = 7.5           # Constant
b    = 237.3         # Constant

length = df["Time"].to_numpy().shape[0]

In [6]:
FT = (rhoa*C*cp)*df["Mean wind speed"]*(df["Air temperature"]-Ts) # Turbulent flux (sensible + latent). Ta-Ts, differensial=>can use degree Celsius
FL = -es*(sig*(Ts*np.ones(length)+273.15)**4)                       # Balckbody radiation form the ocean. Ts+273.15=degree Kelvin
ea = 0.7829*(1+0.2232*(df["Cloud cover"]/8)**(2.75))               # The effective air emissivity
FB = (ea*(sig*(df["Air temperature"]+273.15)**4))                  # Balckbody radiation form the atms. Simonsen & Haugan (1996), in deg. Kelvin
k  = 1-0.6*(df["Cloud cover"]/8)**3                                # Cloud correction term
vp = (df["Relative air humidity"]/100)*6.11*10**(r*df['Air temperature']/(b+df['Air temperature']))       # Vapor pressure

In [7]:
day_of_year = df["Time"].dt.day_of_year
year = df['Time'].dt.year
month = df['Time'].dt.month
day = df['Time'].dt.day
hour = df["Time"].dt.hour
#print(year,month,day,hour,day_of_year)

i  = 23.44*np.cos((360/365)*(172-day_of_year)*2*np.pi/360)
hour_s = hour+2                               #Sun hour = 2 hours behind local time
sun_hour_angle  = (12-hour_s)*15                                  #Sun hour angle

coszp = np.sin(psi*np.pi/180)*np.sin(i*np.pi/180)+np.cos(psi*np.pi/180)*np.cos(i*np.pi/180)*np.cos(sun_hour_angle)
coszp[coszp<0]=0
Q0 = ((S0*coszp**2)/(1.085*coszp+(2.7+coszp)*vp*1e-3+0.1))
FS = (1-alpha)*k*Q0
Fnet = FT+FL-FB-FS

In [8]:
output = pd.DataFrame()
output['time']=df['Time']
output['sw'] = FS
output['lw'] = FL + FB
output['qlat'] = np.zeros(df['Time'].to_numpy().shape)
output['qsens'] = FT
output['tx'] = df['taux']
output['ty'] = df['tauy']
output['precip'] = np.zeros(df['Time'].to_numpy().shape)
output = output.iloc[::3,:]
output['time'] = mdates.date2num(output['time'])-mdates.date2num(output['time'][0])

output_xr = output.to_xarray()
output_xr.to_netcdf(path="../input_data/Svalbard_Lufthavn.nc", mode='w')